# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import sqlalchemy
import numpy as np
import re
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, balanced_accuracy_score

In [2]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///disastermessages.db')
df = pd.read_sql_table("messages_08_06_19_17", con=engine)
X = df.message.values
Y = df.values[:, 6:]

In [3]:
df.columns[6:].__len__()

36

In [4]:
df[df.related == 2.0].values[:, 7:].sum()

0.0

In [5]:
categories = df.columns.values[6:]

### 2. Write a tokenization function to process your text data

In [6]:
wtok = RegexpTokenizer("\w+")
stemmer = SnowballStemmer("english")

In [7]:
def pre_clean(text):
    text = str(text)
    re.sub("[^aA-zZ]", "", text)
    re.sub("-|_|\^", "", text)
    return text    

In [8]:
def tokenize(text):
    text = pre_clean(text)
    wlist = [stemmer.stem(w.lower()) 
                 for w in wtok.tokenize(text) 
                     if w.lower() 
                     not in stopwords.words("english")]
    return wlist

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ("multi", MultiOutputClassifier(RandomForestClassifier(100)))
        #("multi", MultiOutputClassifier(OneVsRestClassifier(SVC(kernel="linear"))))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.33)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
import numpy as np
ytrain = ytrain.astype("int")

In [12]:
pipeline.fit(xtrain, ytrain)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

In [13]:
ypred = pipeline.predict(xtest)

In [14]:
ytest = ytest.astype("int")

In [15]:
accf1 = []
for cat, cat_desc in zip(range(1, 36), categories):
    accf1.append(f1_score(ytest[:, cat], ypred[:, cat]))
    print(f"{cat_desc} \n{classification_report(ytest[:, cat], ypred[:, cat])[:165]}")

related 
              precision    recall  f1-score   support

           0       0.83      0.99      0.90      7165
           1       0.39      0.05      0.08      1484

 
request 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8610
           1       0.00      0.00      0.00        39

 
offer 
              precision    recall  f1-score   support

           0       0.60      0.81      0.69      5076
           1       0.46      0.23      0.31      3573

 
aid_related 
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      7978
           1       0.09      0.01      0.01       671

 
medical_help 
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      8215
           1       0.00      0.00      0.00       434

 
medical_products 
              precision    recall  f1-score   support

           0       0.97      1.00      0.98  

c:\users\manuelmartin\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\manuelmartin\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


death 
              precision    recall  f1-score   support

           0       0.87      0.99      0.93      7528
           1       0.04      0.00      0.01      1121

 
other_aid 
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      8091
           1       0.09      0.00      0.01       558

 
infrastructure_related 
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      8249
           1       0.14      0.01      0.01       400

 
transport 
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      8211
           1       0.00      0.00      0.00       438

 
buildings 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      8466
           1       0.00      0.00      0.00       183

 
electricity 
              precision    recall  f1-score   support

           0       0.99      1.00     

In [16]:
(ypred == ytest).mean()

0.9248178980228928

In [17]:
catmap = {ind : cat for ind, cat in zip(range(0, len(categories)), categories)}

def getcatmap(pred_array):
    indices = list(np.argwhere(pred_array.flatten() == 1).flatten())
    print(indices)
    categories = [catmap[ind] for ind in indices]
    
    if not categories:
        return "not related"
    return categories

In [18]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x000001AD5C12DC80>,
           vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('multi',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
               oob_score=Fals

###### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {"vect__min_df" : [1, 3]}

cv = GridSearchCV(pipeline, parameters, iid=True, cv=2)
cv.fit(xtrain, ytrain)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid=True, n_jobs=None,
       param_grid={'vect__min_df': [1, 3]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [22]:
import pickle
with open(r"models/model.pkl", "wb") as f:
    pickle.dump(cv, f)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
ypred = cv.predict(xtest)
(ypred == ytest).mean()

0.9247472411711052

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.